In [57]:
import torch
import os
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, Subset
from torchvision import datasets
from torchvision import transforms, models, utils
from torchvision.transforms import ToTensor
import numpy as np 

In [58]:
import azureml.core
import logging
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

In [59]:
ws = Workspace("4b134a83-eb22-46e0-b44d-696615a1e373", "GSN", "GSN-Projekt")

In [60]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [61]:
import matplotlib.pyplot as plt
import numpy as np

In [62]:
import torch.nn as nn
from torch import optim
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import random
import torch.nn.functional as F
import random
from PIL import Image
from torch.optim import lr_scheduler
import copy
import time
from sklearn.model_selection import train_test_split
plt.ion()

In [33]:
!wget https://github.com/WawrzynP/Constellation_Classification/blob/master/DataSet_JPG.zip?raw=true -O DataSet_JPG.zip

--2022-01-13 22:31:18--  https://github.com/WawrzynP/Constellation_Classification/blob/master/DataSet_JPG.zip?raw=true
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/WawrzynP/Constellation_Classification/raw/master/DataSet_JPG.zip [following]
--2022-01-13 22:31:19--  https://github.com/WawrzynP/Constellation_Classification/raw/master/DataSet_JPG.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/WawrzynP/Constellation_Classification/master/DataSet_JPG.zip [following]
--2022-01-13 22:31:19--  https://raw.githubusercontent.com/WawrzynP/Constellation_Classification/master/DataSet_JPG.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com 

In [4]:
!unzip DataSet_JPG.zip

Archive:  DataSet_JPG.zip
replace DataSet_JPG/Andromeda/img_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [63]:
directory = 'DataSet_JPG'

In [64]:
batch_size = 32
image_size = 256

In [65]:
transform = transforms.Compose(
  [transforms.Resize([image_size, image_size]),
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])
data = datasets.ImageFolder(root=directory, transform=transform)
classNames = data.classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [66]:
def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

datasets = train_val_dataset(data)
dataloaders = {x:DataLoader(datasets[x],32, shuffle=True, num_workers=4) for x in ['train','val']}
x,y = next(iter(dataloaders['train']))

In [67]:
dataset_sizes = {x: len(datasets[x]) for x in ['train', 'val']}
class_names = data.classes

In [68]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    val_acc_history = []
    val_loss_history = []
    train_acc_history = []
    train_loss_history = []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
                val_loss_history.append(epoch_loss)
            if phase == 'train':
                train_acc_history.append(epoch_acc)
                train_loss_history.append(epoch_loss)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, val_loss_history, train_acc_history, train_loss_history

In [69]:
def plot_acc_loss(val_acc_history, val_loss_history, train_acc_history, train_loss_history):
  val_acc_hist = []
  acc_hist = [h.cpu().numpy() for h in val_acc_history]
  train_acc_hist = []
  t_acc_hist = [h.cpu().numpy() for h in train_acc_history]
  val_loss_history
  num_epoch = 3

  plt.figure(figsize=(12, 9))
  plt.plot(
          t_acc_hist, color='green', linestyle='-', 
          label='train accuracy')

  plt.plot(
          acc_hist, color='blue', linestyle='-', 
          label='validataion accuracy')
  plt.title("Validation Accuracy")
  plt.xlabel("Training Epochs")
  plt.ylabel("Accuracy")
  plt.legend()
  plt.show()

  plt.figure(figsize=(12, 9))
  plt.plot(
          train_loss_history, color='orange', linestyle='-', 
          label='train loss')
  plt.plot(
          val_loss_history, color='red', linestyle='-', 
          label='validataion loss')
  plt.xlabel('Training Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

In [70]:
def conf_matrix(model, net_name):
    model.eval()
    confusion_matrix = torch.zeros(len(classNames), len(classNames))
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for l, p in zip(labels.view(-1), preds.view(-1)):
              confusion_matrix[l.long(), p.long()] += 1

        plt.figure(figsize = (20,20))
        sn.heatmap(confusion_matrix, annot=True, cmap="Blues", xticklabels=classNames, yticklabels=classNames)
        plt.title('{name} \nAccuracy:{num:.3f}'.format(name=net_name, num=np.trace(confusion_matrix)/2816), fontdict={'size':'21'})
        plt.ylabel('Actual label', fontdict={'size':'18'})
        plt.xlabel('Predicted label', fontdict={'size':'18'})
        plt.show()

In [16]:
# ResNet18

In [17]:
model_ft_rs18 = models.resnet18(pretrained=True)
num_ft_rs18 = model_ft_rs18.fc.in_features
model_ft_rs18.fc = nn.Linear(num_ft_rs18, len(classNames))
model_ft_rs18 = model_ft_rs18.to(device)
criterion_rs18 = nn.CrossEntropyLoss()
optimizer_ft_rs18 = optim.SGD(model_ft_rs18.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler_rs18 = lr_scheduler.StepLR(optimizer_ft_rs18, step_size=7, gamma=0.1)

In [18]:
model_ft_rs18, val_acc_history_rs18, val_loss_history_rs18, train_acc_history_rs18, train_loss_history_rs18 = train_model(model_ft_rs18, criterion_rs18, optimizer_ft_rs18, exp_lr_scheduler_rs18,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 4.3049 Acc: 0.0452
val Loss: 4.0317 Acc: 0.0753

Epoch 1/24
----------
train Loss: 3.4363 Acc: 0.2197
val Loss: 3.5326 Acc: 0.1722

Epoch 2/24
----------
train Loss: 2.6344 Acc: 0.4039
val Loss: 2.6876 Acc: 0.3714

Epoch 3/24
----------
train Loss: 1.9851 Acc: 0.5606
val Loss: 2.3311 Acc: 0.4464

Epoch 4/24
----------
train Loss: 1.4925 Acc: 0.6709
val Loss: 1.8824 Acc: 0.5476

Epoch 5/24
----------
train Loss: 1.1166 Acc: 0.7563
val Loss: 1.6585 Acc: 0.5877

Epoch 6/24
----------
train Loss: 0.8355 Acc: 0.8196
val Loss: 1.2688 Acc: 0.6722

Epoch 7/24
----------
train Loss: 0.6142 Acc: 0.8930
val Loss: 0.9446 Acc: 0.7532

Epoch 8/24
----------
train Loss: 0.5731 Acc: 0.9061
val Loss: 0.9364 Acc: 0.7532

Epoch 9/24
----------
train Loss: 0.5544 Acc: 0.9106
val Loss: 0.9177 Acc: 0.7582

Epoch 10/24
----------
train Loss: 0.5324 Acc: 0.9167
val Loss: 0.9153 Acc: 0.7550

Epoch 11/24
----------
train Loss: 0.5108 Acc: 0.9241
val Loss: 0.8948 Acc: 0.7656

Ep

In [ ]:
conf_matrix(model_ft_rs18, 'ResNet18')

In [ ]:
plot_acc_loss(val_acc_history_rs18, val_loss_history_rs18, train_acc_history_rs18, train_loss_history_rs18)

In [42]:
# ResNet50

In [43]:
model_ft_rs50 = models.resnet50(pretrained=True)
num_ft_rs50 = model_ft_rs50.fc.in_features

model_ft_rs50.fc = nn.Linear(num_ft_rs50, len(classNames))

model_ft_rs50 = model_ft_rs50.to(device)

criterion_rs50 = nn.CrossEntropyLoss()

optimizer_ft_rs50 = optim.SGD(model_ft_rs50.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler_rs50 = lr_scheduler.StepLR(optimizer_ft_rs50, step_size=7, gamma=0.1)

In [44]:
model_ft_rs50, val_acc_history_rs50, val_loss_history_rs50, train_acc_history_rs50, train_loss_history_rs50 = train_model(model_ft_rs50, criterion_rs50, optimizer_ft_rs50, exp_lr_scheduler_rs50,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 4.3059 Acc: 0.0478
val Loss: 4.0532 Acc: 0.0774

Epoch 1/24
----------
train Loss: 3.5070 Acc: 0.1931
val Loss: 3.3143 Acc: 0.2077

Epoch 2/24
----------
train Loss: 2.6475 Acc: 0.3642
val Loss: 3.0324 Acc: 0.2553

Epoch 3/24
----------
train Loss: 1.9188 Acc: 0.5368
val Loss: 2.1667 Acc: 0.4368

Epoch 4/24
----------
train Loss: 1.3606 Acc: 0.6612
val Loss: 1.7813 Acc: 0.5252

Epoch 5/24
----------
train Loss: 0.9572 Acc: 0.7620
val Loss: 2.6735 Acc: 0.3782

Epoch 6/24
----------
train Loss: 0.6638 Acc: 0.8355
val Loss: 1.1640 Acc: 0.6538

Epoch 7/24
----------
train Loss: 0.4445 Acc: 0.9096
val Loss: 0.7494 Acc: 0.7695

Epoch 8/24
----------
train Loss: 0.4021 Acc: 0.9258
val Loss: 0.7350 Acc: 0.7759

Epoch 9/24
----------
train Loss: 0.3723 Acc: 0.9325
val Loss: 0.7269 Acc: 0.7752

Epoch 10/24
----------
train Loss: 0.3552 Acc: 0.9411
val Loss: 0.7201 Acc: 0.7720

Epoch 11/24
----------
train Loss: 0.3317 Acc: 0.9446
val Loss: 0.7047 Acc: 0.7820

Ep

In [ ]:
conf_matrix(model_ft_rs50, 'ResNet50')

In [ ]:
plot_acc_loss(val_acc_history_rs50, val_loss_history_rs50, train_acc_history_rs50, train_loss_history_rs50)

In [71]:
# ResNet101

In [72]:
model_ft_rs101 = models.resnet101(pretrained=True)
num_ft_rs101 = model_ft_rs101.fc.in_features

model_ft_rs101.fc = nn.Linear(num_ft_rs101, len(classNames))

model_ft_rs101 = model_ft_rs101.to(device)

criterion_rs101 = nn.CrossEntropyLoss()

optimizer_ft_rs101 = optim.SGD(model_ft_rs101.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler_rs101 = lr_scheduler.StepLR(optimizer_ft_rs101, step_size=7, gamma=0.1)

In [74]:
model_ft_rs101, val_acc_history_rs101, val_loss_history_rs101, train_acc_history_rs101, train_loss_history_rs101= train_model(model_ft_rs101, criterion_rs101, optimizer_ft_rs101, exp_lr_scheduler_rs101,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 0.1846 Acc: 0.9762
val Loss: 0.6108 Acc: 0.8022

Epoch 1/24
----------
train Loss: 0.1787 Acc: 0.9768
val Loss: 0.6081 Acc: 0.8001

Epoch 2/24
----------
train Loss: 0.1796 Acc: 0.9773
val Loss: 0.6020 Acc: 0.8029

Epoch 3/24
----------
train Loss: 0.1831 Acc: 0.9774
val Loss: 0.6008 Acc: 0.8058

Epoch 4/24
----------
train Loss: 0.1828 Acc: 0.9760
val Loss: 0.6071 Acc: 0.8029

Epoch 5/24
----------
train Loss: 0.1813 Acc: 0.9749
val Loss: 0.5998 Acc: 0.8058

Epoch 6/24
----------
train Loss: 0.1812 Acc: 0.9767
val Loss: 0.6047 Acc: 0.8068

Epoch 7/24
----------
train Loss: 0.1795 Acc: 0.9762
val Loss: 0.6019 Acc: 0.8050

Epoch 8/24
----------
train Loss: 0.1782 Acc: 0.9774
val Loss: 0.6055 Acc: 0.8008

Epoch 9/24
----------
train Loss: 0.1789 Acc: 0.9782
val Loss: 0.6049 Acc: 0.8004

Epoch 10/24
----------
train Loss: 0.1804 Acc: 0.9768
val Loss: 0.6074 Acc: 0.8054

Epoch 11/24
----------
train Loss: 0.1797 Acc: 0.9766
val Loss: 0.6034 Acc: 0.7987

Ep

In [ ]:
conf_matrix(model_ft_rs101, 'ResNet101')

In [ ]:
plot_acc_loss(val_acc_history_rs101, val_loss_history_rs101, train_acc_history_rs101, train_loss_history_rs101)

In [75]:
# VGG11

In [76]:
model_ft_vgg11 = models.vgg11_bn(pretrained=True)
num_ft_vgg11 = model_ft_vgg11.classifier[6].in_features
model_ft_vgg11.classifier[6] = nn.Linear(num_ft_vgg11, len(classNames))

model_ft_vgg11 = model_ft_vgg11.to(device)
criterion_vgg11 = nn.CrossEntropyLoss()
optimizer_ft_vgg11 = optim.SGD(model_ft_vgg11.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler_vgg11 = lr_scheduler.StepLR(optimizer_ft_vgg11, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/vgg11_bn-6002323d.pth" to /home/azureuser/.cache/torch/hub/checkpoints/vgg11_bn-6002323d.pth


  0%|          | 0.00/507M [00:00<?, ?B/s]

In [77]:
model_ft_vgg11, val_acc_history_vgg11, val_loss_history_vgg11, train_acc_history_vgg11, train_loss_history_vgg11= train_model(model_ft_vgg11, criterion_vgg11, optimizer_ft_vgg11, exp_lr_scheduler_vgg11,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 4.1898 Acc: 0.0586
val Loss: 3.7159 Acc: 0.1346

Epoch 1/24
----------
train Loss: 3.2505 Acc: 0.1972
val Loss: 2.8029 Acc: 0.2898

Epoch 2/24
----------
train Loss: 2.4311 Acc: 0.3572
val Loss: 2.3278 Acc: 0.3761

Epoch 3/24
----------
train Loss: 1.8696 Acc: 0.4859
val Loss: 1.8539 Acc: 0.4648

Epoch 4/24
----------
train Loss: 1.4496 Acc: 0.5889
val Loss: 1.4900 Acc: 0.5604

Epoch 5/24
----------
train Loss: 1.1428 Acc: 0.6628
val Loss: 1.4194 Acc: 0.5820

Epoch 6/24
----------
train Loss: 0.8906 Acc: 0.7337
val Loss: 1.3274 Acc: 0.6097

Epoch 7/24
----------
train Loss: 0.5821 Acc: 0.8280
val Loss: 0.9239 Acc: 0.7081

Epoch 8/24
----------
train Loss: 0.5053 Acc: 0.8518
val Loss: 0.9039 Acc: 0.7138

Epoch 9/24
----------
train Loss: 0.4758 Acc: 0.8552
val Loss: 0.8911 Acc: 0.7166

Epoch 10/24
----------
train Loss: 0.4619 Acc: 0.8658
val Loss: 0.8816 Acc: 0.7195

Epoch 11/24
----------
train Loss: 0.4320 Acc: 0.8723
val Loss: 0.8775 Acc: 0.7209

Ep

In [ ]:
conf_matrix(model_ft_vgg11, 'VGG11')

In [ ]:
plot_acc_loss(val_acc_history_vgg11, val_loss_history_vgg11, train_acc_history_vgg11, train_loss_history_vgg11)

In [78]:
# Densenet121

In [79]:
model_ft_ds121 = models.densenet121(pretrained=True)
num_ft_ds121 = model_ft_ds121.classifier.in_features
model_ft_ds121.classifier = nn.Linear(num_ft_ds121, len(classNames))

model_ft_ds121 = model_ft_ds121.to(device)
criterion_ds121 = nn.CrossEntropyLoss()
optimizer_ft_ds121 = optim.SGD(model_ft_ds121.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler_ds121 = lr_scheduler.StepLR(optimizer_ft_ds121, step_size=7, gamma=0.1)

In [80]:
model_ft_ds121, val_acc_history_ds121, val_loss_history_ds121, train_acc_history_ds121, train_loss_history_ds121= train_model(model_ft_ds121, criterion_ds121, optimizer_ft_ds121, exp_lr_scheduler_ds121,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 4.3411 Acc: 0.0425
val Loss: 4.0306 Acc: 0.0987

Epoch 1/24
----------
train Loss: 3.5070 Acc: 0.1937
val Loss: 3.2139 Acc: 0.2319

Epoch 2/24
----------
train Loss: 2.5754 Acc: 0.4009
val Loss: 2.2321 Acc: 0.4549

Epoch 3/24
----------
train Loss: 1.8507 Acc: 0.5585
val Loss: 2.1575 Acc: 0.4659

Epoch 4/24
----------
train Loss: 1.3448 Acc: 0.6694
val Loss: 1.6010 Acc: 0.5774

Epoch 5/24
----------
train Loss: 0.9619 Acc: 0.7595
val Loss: 0.9813 Acc: 0.7230

Epoch 6/24
----------
train Loss: 0.7093 Acc: 0.8213
val Loss: 0.9813 Acc: 0.7120

Epoch 7/24
----------
train Loss: 0.4878 Acc: 0.8949
val Loss: 0.7107 Acc: 0.7798

Epoch 8/24
----------
train Loss: 0.4355 Acc: 0.9134
val Loss: 0.6901 Acc: 0.7855

Epoch 9/24
----------
train Loss: 0.4106 Acc: 0.9214
val Loss: 0.6840 Acc: 0.7820

Epoch 10/24
----------
train Loss: 0.3859 Acc: 0.9279
val Loss: 0.6624 Acc: 0.7923

Epoch 11/24
----------
train Loss: 0.3738 Acc: 0.9319
val Loss: 0.6531 Acc: 0.7969

Ep

In [ ]:
conf_matrix(model_ft_ds121, 'Densenet')

In [ ]:
plot_acc_loss(val_acc_history_ds121, val_loss_history_ds121, train_acc_history_ds121, train_loss_history_ds121)

In [81]:
# Squeezenet

In [82]:
model_ft_sqn = models.squeezenet1_0(pretrained=True)
model_ft_sqn.classifier[1] = nn.Conv2d(512, len(classNames), kernel_size=(1,1), stride=(1,1))
model_ft_sqn.num_classes = len(classNames)

model_ft_sqn = model_ft_sqn.to(device)
criterion_sqn = nn.CrossEntropyLoss()
optimizer_ft_sqn = optim.SGD(model_ft_sqn.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler_sqn = lr_scheduler.StepLR(optimizer_ft_sqn, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/squeezenet1_0-a815701f.pth" to /home/azureuser/.cache/torch/hub/checkpoints/squeezenet1_0-a815701f.pth


  0%|          | 0.00/4.79M [00:00<?, ?B/s]

In [83]:
model_ft_sqn, val_acc_history_sqn, val_loss_history_sqn, train_acc_history_sqn, train_loss_history_sqn = train_model(model_ft_sqn, criterion_sqn, optimizer_ft_sqn, exp_lr_scheduler_sqn,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 4.4559 Acc: 0.0144
val Loss: 4.4459 Acc: 0.0163

Epoch 1/24
----------
train Loss: 4.3568 Acc: 0.0256
val Loss: 4.2672 Acc: 0.0437

Epoch 2/24
----------
train Loss: 4.1881 Acc: 0.0476
val Loss: 4.2067 Acc: 0.0455

Epoch 3/24
----------
train Loss: 4.0090 Acc: 0.0723
val Loss: 3.8685 Acc: 0.1143

Epoch 4/24
----------
train Loss: 3.7288 Acc: 0.1143
val Loss: 3.5847 Acc: 0.1420

Epoch 5/24
----------
train Loss: 3.4085 Acc: 0.1636
val Loss: 3.2739 Acc: 0.1946

Epoch 6/24
----------
train Loss: 3.1159 Acc: 0.2205
val Loss: 3.0407 Acc: 0.2436

Epoch 7/24
----------
train Loss: 2.5089 Acc: 0.3471
val Loss: 2.3846 Acc: 0.3647

Epoch 8/24
----------
train Loss: 2.3209 Acc: 0.3899
val Loss: 2.3257 Acc: 0.3796

Epoch 9/24
----------
train Loss: 2.2294 Acc: 0.4041
val Loss: 2.2545 Acc: 0.3970

Epoch 10/24
----------
train Loss: 2.1801 Acc: 0.4231
val Loss: 2.1730 Acc: 0.4155

Epoch 11/24
----------
train Loss: 2.0957 Acc: 0.4369
val Loss: 2.1574 Acc: 0.4041

Ep

In [ ]:
conf_matrix(model_ft_sqn, 'Squeezenet')

In [ ]:
plot_acc_loss(val_acc_history_sqn, val_loss_history_sqn, train_acc_history_sqn, train_loss_history_sqn)